In [4]:
import os
import requests

In [8]:
url = "https://apis.data.go.kr/B551011/KorService2/locationBasedList2"
serviceKey = "ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg=="
base_path = "./image"

# 2. 사용자가 정의한 관광지 리스트 (폴더명)
target_locations = [
    "영화의전당", "호천마을", "국제시장", "화명생태공원", "동래읍성", 
    "벡스코", "전포카페거리", "자갈치시장", "문화빙상센터", "동래온천", 
    "더베이101", "부전마켓타운", "비프광장", "구포시장", "금강공원", 
    "누리마루", "송상현광장", "용두산공원", "부산어촌민속관", 
    "해운대해수욕장", "황령산", "보수동책방골목", "해운대블루라인파크", 
    "성지곡수원지", "장산", "송정해수욕장"
]

params = {
    'serviceKey': serviceKey,
    'numOfRows': '100',  # 더 많은 데이터를 위해 증가
    'pageNo': '1',
    'MobileOS': 'ETC',
    'MobileApp': 'AppTest',
    '_type': 'json',
    'arrange': 'O',
    'mapX': '129.1585', 
    'mapY': '35.1587',
    'radius': '20000', # 부산 전역을 커버하기 위해 20km로 확대
    'contentTypeId': '12'
}

In [2]:
serviceKey = "ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg=="
list_url = "https://apis.data.go.kr/B551011/KorService1/locationBasedList1"
detail_img_url = "https://apis.data.go.kr/B551011/KorService1/detailImage1"

In [5]:
response = requests.get(url, params=params)
data = response.json()

try:
    items_container = data.get('response', {}).get('body', {}).get('items')
    
    if items_container and items_container != "":
        items = items_container.get('item')
        if isinstance(items, dict): items = [items]
        
        print(f"--- 고유 ID 기반 이미지 저장 시작 ---\n")
        
        for item in items:
            title = item.get('title')
            img_url = item.get('firstimage')
            content_id = item.get('contentid') # API가 제공하는 고유 번호
            
            if not img_url or not content_id: continue

            # 폴더 매칭
            matched_folder = None
            for loc in target_locations:
                if loc in title:
                    matched_folder = loc
                    break
            
            if matched_folder:
                save_dir = os.path.join(base_path, matched_folder)
                if not os.path.exists(save_dir): os.makedirs(save_dir)
                
                # 고유 식별자(content_id)를 파일명에 사용 (중복 방지의 핵심)
                file_name = f"{matched_folder}_{content_id}.jpg"
                save_path = os.path.join(save_dir, file_name)
                
                # 이미 파일이 존재하면 건너뛰기 (선택 사항: 업데이트를 원하면 이 조건을 제거)
                if os.path.exists(save_path):
                    print(f"이미 존재함 (Pass): {file_name}")
                    continue

                try:
                    img_data = requests.get(img_url, timeout=10).content
                    with open(save_path, 'wb') as f:
                        f.write(img_data)
                    print(f"저장 성공: {file_name}")
                except Exception as e:
                    print(f"다운로드 실패: {title} ({e})")
    else:
        print("데이터가 없습니다.")

except Exception as e:
    print(f"처리 중 오류 발생: {e}")

--- 고유 ID 기반 이미지 저장 시작 ---

이미 존재함 (Pass): 국제시장_1018702.jpg
이미 존재함 (Pass): 금강공원_127445.jpg
이미 존재함 (Pass): 더베이101_2350092.jpg
이미 존재함 (Pass): 동래온천_2822334.jpg
이미 존재함 (Pass): 동래읍성_2654613.jpg
이미 존재함 (Pass): 벡스코_3083429.jpg


In [6]:
import requests
import os
import time

# 1. 설정
serviceKey = "ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg=="
base_path = "./image"

# 우리가 딱 필요한 장소들
target_locations = [
    "영화의전당", "호천마을", "국제시장", "화명생태공원", "동래읍성", 
    "벡스코", "전포카페거리", "자갈치시장", "문화빙상센터", "동래온천", 
    "더베이101", "부전마켓타운", "비프광장", "구포시장", "금강공원", 
    "누리마루", "송상현광장", "용두산공원", "부산어촌민속관", 
    "해운대해수욕장", "황령산", "보수동책방골목", "해운대블루라인파크", 
    "성지곡수원지", "장산", "송정해수욕장"
]

def collect_target_data(mapX, mapY):
    list_url = "https://apis.data.go.kr/B551011/KorService2/locationBasedList2"
    params = {
        'serviceKey': serviceKey,
        'numOfRows': '100', # 한 번에 많이 훑기
        'MobileOS': 'ETC',
        'MobileApp': 'AppTest',
        '_type': 'json',
        'mapX': mapX,
        'mapY': mapY,
        'radius': '10000', # 반경 10km
        'contentTypeId': '12'
    }
    
    try:
        res = requests.get(list_url, params=params).json()
        items = res.get('response', {}).get('body', {}).get('items', {}).get('item', [])
        if isinstance(items, dict): items = [items]

        for item in items:
            api_title = item.get('title', '')
            contentId = item.get('contentid')

            # [핵심] API 제목에 우리 타겟 리스트의 이름이 있는지 확인
            matched_loc = next((loc for loc in target_locations if loc in api_title), None)
            
            if matched_loc:
                print(f"🎯 타겟 발견: [{matched_loc}] (API 명칭: {api_title})")
                fetch_and_save_gallery(contentId, matched_loc)
                time.sleep(0.3)
    except Exception as e:
        print(f"❌ 오류: {e}")

def fetch_and_save_gallery(contentId, folder_name):
    # 상세 이미지(갤러리) 조회
    detail_url = "https://apis.data.go.kr/B551011/KorService2/detailImage2"
    params = {
        'serviceKey': serviceKey,
        'MobileOS': 'ETC',
        'MobileApp': 'AppTest',
        '_type': 'json',
        'contentId': contentId,
        'imageMsType': '1'
    }
    
    try:
        res = requests.get(detail_url, params=params).json()
        img_items = res.get('response', {}).get('body', {}).get('items', {}).get('item', [])
        if isinstance(img_items, dict): img_items = [img_items]

        # 폴더 생성
        save_dir = os.path.join(base_path, folder_name)
        if not os.path.exists(save_dir): os.makedirs(save_dir)

        for idx, img in enumerate(img_items):
            url = img.get('originimgurl')
            if url:
                img_data = requests.get(url).content
                file_name = f"{contentId}_{idx}.jpg"
                with open(os.path.join(save_dir, file_name), 'wb') as f:
                    f.write(img_data)
                print(f"   ㄴ 이미지 저장: {folder_name}/{file_name}")
    except:
        pass

# 주요 거점 스캔 (해운대, 남포동, 서면, 동래, 화명)
points = [('129.1585', '35.1587'), ('129.0401', '35.1051'), ('129.0595', '35.1555'), ('129.0838', '35.2048'), ('129.0121', '35.2358')]

for mx, my in points:
    collect_target_data(mx, my)

🎯 타겟 발견: [해운대해수욕장] (API 명칭: 해운대해수욕장)
🎯 타겟 발견: [더베이101] (API 명칭: 더베이101)
🎯 타겟 발견: [벡스코] (API 명칭: 벡스코 상상체험 키즈월드)
🎯 타겟 발견: [장산] (API 명칭: 장산 (부산 국가지질공원))
🎯 타겟 발견: [영화의전당] (API 명칭: 영화의전당 라이브러리 자료실)
🎯 타겟 발견: [송정해수욕장] (API 명칭: 송정해수욕장)
🎯 타겟 발견: [동래온천] (API 명칭: 동래온천길(온천천 카페거리))
🎯 타겟 발견: [황령산] (API 명칭: 황령산)
🎯 타겟 발견: [황령산] (API 명칭: 황령산 전망대)
🎯 타겟 발견: [황령산] (API 명칭: 황령산레포츠공원)
🎯 타겟 발견: [동래읍성] (API 명칭: 동래읍성지)
🎯 타겟 발견: [용두산공원] (API 명칭: 용두산공원)
🎯 타겟 발견: [국제시장] (API 명칭: 국제시장 먹자골목)
🎯 타겟 발견: [호천마을] (API 명칭: 호천마을)
🎯 타겟 발견: [전포카페거리] (API 명칭: 전포카페거리)
🎯 타겟 발견: [송상현광장] (API 명칭: 송상현광장)
🎯 타겟 발견: [황령산] (API 명칭: 황령산레포츠공원)
🎯 타겟 발견: [호천마을] (API 명칭: 호천마을)
🎯 타겟 발견: [황령산] (API 명칭: 황령산)
🎯 타겟 발견: [황령산] (API 명칭: 황령산 전망대)
🎯 타겟 발견: [성지곡수원지] (API 명칭: 성지곡수원지)
🎯 타겟 발견: [동래온천] (API 명칭: 동래온천길(온천천 카페거리))
🎯 타겟 발견: [동래읍성] (API 명칭: 동래읍성지)
🎯 타겟 발견: [동래온천] (API 명칭: 동래온천 노천족탕)
🎯 타겟 발견: [금강공원] (API 명칭: 금강공원)
🎯 타겟 발견: [동래온천] (API 명칭: 동래온천길(온천천 카페거리))
🎯 타겟 발견: [성지곡수원지] (API 명칭: 성지곡수원지)
🎯 타겟 발견: [송상현광장] (API 명칭: 송상현광장)
🎯 타겟 발견: [황령산] (API 명

In [7]:
import requests
import os

# 1. 초기 세팅
serviceKey = "ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg=="
# 빵천동(남천동)의 공식 contentId
target_id = "2545417"
save_path = "./image/빵천동"

if not os.path.exists(save_path):
    os.makedirs(save_path)

# 2. detailImage2 URL 조립
url = "https://apis.data.go.kr/B551011/KorService2/detailImage2"
params = {
    'serviceKey': serviceKey,
    'MobileOS': 'ETC',
    'MobileApp': 'AppTest',
    '_type': 'json',
    'contentId': target_id,
    'imageMsType': '1' # 갤러리 이미지 전체
}

# 3. 데이터 요청 및 저장
try:
    response = requests.get(url, params=params)
    data = response.json()
    
    items_container = data.get('response', {}).get('body', {}).get('items')
    
    if items_container and items_container != "":
        items = items_container.get('item', [])
        if isinstance(items, dict): items = [items]
        
        print(f"🍞 빵천동 이미지 수집 시작 (총 {len(items)}장 발견)")
        
        for idx, item in enumerate(items):
            img_url = item.get('originimgurl')
            if img_url:
                img_data = requests.get(img_url).content
                with open(f"{save_path}/bbang_{idx}.jpg", 'wb') as f:
                    f.write(img_data)
                print(f"   ㄴ {idx+1}번 이미지 저장 완료")
    else:
        print("❌ 해당 ID에 등록된 갤러리 이미지가 없습니다.")

except Exception as e:
    print(f"❗ 오류 발생: {e}")

❌ 해당 ID에 등록된 갤러리 이미지가 없습니다.
